In [27]:
subdir_name = 'pdfs-OCR-22_1030/' # Include / as last char
core_name = 'Comparative Salaries for Cities' # Use to Infer name.pdf output and also .csv filenames
file_loc_in_bucket = 'jpg_organized/' + subdir_name + core_name + '.pdf'

In [28]:
import boto3
import textractor
from textractprettyprinter.t_pretty_print import Pretty_Print_Table_Format, Textract_Pretty_Print, get_string

In [29]:
# Config
REGION = aws_region = boto3.session.Session().region_name # 'us-west-2'
BUCKET = bucket_name = 'andrew-ocr'

In [30]:
s3 = boto3.client('s3', REGION)
textract = boto3.client('textract', aws_region) 

In [31]:
"""For reference"""
# def analyze_pdf_document_with_a2i(document_name, bucket):
#     response = textract.start_document_analysis(
#         DocumentLocation={'S3Object': {'Bucket': bucket, 'Name': document_name}},
#         FeatureTypes=["TABLES", "FORMS"], 
# #         HumanLoopConfig=humanLoopConfig
#     )
#     return response

'For reference'

In [32]:
analyzePDF = textract.start_document_analysis(
    DocumentLocation={'S3Object': {'Bucket': bucket_name, 'Name': file_loc_in_bucket}},
    FeatureTypes=["TABLES"] # Careful! TABLES, FORMS
)

analyzePDF_results= textract.get_document_analysis(
    JobId=analyzePDF['JobId']
)

In [33]:
def GetResults(jobId):
    maxResults = 1000
    paginationToken = None
    finished = False
    pages= []
    while finished == False:
        response = None
#         if self.processType == ProcessType.ANALYSIS:
        if paginationToken == None:
#             response = self.textract.get_document_analysis(JobId=jobId, MaxResults=maxResults)
            response = textract.get_document_analysis(JobId=jobId)
        else:
#             response = self.textract.get_document_analysis(JobId=jobId, MaxResults=maxResults, NextToken=paginationToken)
            response = textract.get_document_analysis(JobId=jobId, NextToken=paginationToken)
#         if self.processType == ProcessType.DETECTION:
#            if paginationToken == None:
#                response = self.textract.get_document_text_detection(JobId=jobId, MaxResults=maxResults)
#            else:
#                response = self.textract.get_document_text_detection(JobId=jobId,MaxResults=maxResults, NextToken=paginationToken)
        blocks = response['Blocks']
        print('Detected Document Text')
        print('Pages: {}'.format(response['DocumentMetadata']['Pages']))
        # Display block information
#         for block in blocks:
#            self.DisplayBlockInfo(block)
#            print()
#            print()
        if 'NextToken' in response:
           paginationToken = response['NextToken']
        else:
            finished = True
#             return response
    # Put response on pages list
        pages.append(response)
        print('Document Detected.')

        if 'NextToken' in response:
            paginationToken = response['NextToken']
        else:
            finished = True

    # convert pages as JSON pattern
    pages = json.dumps(pages)
    return pages



# Note: 
You may have to wait for results to finsh processing (in the cloud) before executing the rest of the cells. 

In [37]:
analyzePDF

{'JobId': '6a4fc1f2cfa1d05924eae63ee62efb45d0e5c63df03df4c9c1071b2e78fa1156',
 'ResponseMetadata': {'RequestId': '3e929a7d-2cf2-4c78-a8b8-9bcf47f4dff0',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '3e929a7d-2cf2-4c78-a8b8-9bcf47f4dff0',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '76',
   'date': 'Sat, 08 Apr 2023 02:02:41 GMT'},
  'RetryAttempts': 0}}

In [38]:
from time import sleep
sleep(10)

In [39]:
JOBID = analyzePDF['JobId']
analyzePDF_results_all = GetResults(JOBID)
outputInJsonText= str(analyzePDF_results_all)

output_subdir = 'output-OCR-22_1030'
output_filename = output_subdir + core_name + '.json'
s3.put_object(Body=outputInJsonText, Bucket=BUCKET, Key=output_filename)

Detected Document Text
Pages: 12
Document Detected.
Detected Document Text
Pages: 12
Document Detected.
Detected Document Text
Pages: 12
Document Detected.
Detected Document Text
Pages: 12
Document Detected.
Detected Document Text
Pages: 12
Document Detected.
Detected Document Text
Pages: 12
Document Detected.


{'ResponseMetadata': {'RequestId': 'RSS6R848W3N8YDJ2',
  'HostId': 'gYZcumfdbRNtHx3Wc0oFchDN22sO4wp3ooB4NB+BBNXRacdb47HTRU8nsYXjnG5hzX0VOmUtkjQ=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'gYZcumfdbRNtHx3Wc0oFchDN22sO4wp3ooB4NB+BBNXRacdb47HTRU8nsYXjnG5hzX0VOmUtkjQ=',
   'x-amz-request-id': 'RSS6R848W3N8YDJ2',
   'date': 'Sat, 08 Apr 2023 02:03:24 GMT',
   'x-amz-version-id': 'bviAaHfqckX1hAp2Ww7fQZYltXoUsTiR',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"90352d43d40d9ded9b41062bca6e8bf4"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"90352d43d40d9ded9b41062bca6e8bf4"',
 'ServerSideEncryption': 'AES256',
 'VersionId': 'bviAaHfqckX1hAp2Ww7fQZYltXoUsTiR'}

In [40]:
output = s3.get_object(Bucket=BUCKET, Key=output_filename)

streaming_obj = output['Body']
string_data = streaming_obj.read().decode("utf-8")

json_data= json.loads(string_data)

In [41]:
textract_json = json_data
textractor_csv = get_string(textract_json=textract_json,
               table_format=Pretty_Print_Table_Format.csv,
               output_type=[Textract_Pretty_Print.TABLES, Textract_Pretty_Print.FORMS])

In [42]:
prefix= '../csvs-OCR-22_1030/'
output_file = prefix + core_name +'.csv'

# replace content
with open(output_file, "wt") as fout:
    fout.write(textractor_csv)